# Finetune CDCP dataset for ACC

## Libraries

In [1]:
# Run this cell only once to install LLaMA-Factory

# %cd ..
# %rm -rf LLaMA-Factory
# !git clone https://github.com/hiyouga/LLaMA-Factory.git
# %cd LLaMA-Factory
# %ls
# !pip install -e .[torch,bitsandbytes]

In [2]:
# !pip uninstall -y pydantic
# !pip install pydantic==1.10.9 # 

# !pip uninstall -y gradio
# !pip install gradio==3.48.0

# !pip uninstall -y bitsandbytes
# !pip install --upgrade bitsandbytes

# !pip install tqdm
# !pip install ipywidgets
# !pip install scikit-learn

# Restart kernel afterwards.

In [1]:
import os
import ast
import sys
import json
import torch
import pickle
import subprocess

sys.path.append('../')

import pandas as pd

import copy
import datasets
from datasets import load_dataset

from tqdm.notebook import tqdm
from llamafactory.chat import ChatModel
from llamafactory.extras.misc import torch_gc
from sklearn.metrics import classification_report
from utils.post_processing import post_process_acc

In [2]:
try:    
    assert torch.cuda.is_available() is True
    
except AssertionError:
    
    print("Please set up a GPU before using LLaMA Factory...")

## Parameters

In [3]:
cdcp_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

In [4]:
cdcp_dir

'/Utilisateurs/umushtaq/am_work/coling_2025/cdcp'

In [5]:
BASE_MODEL = "unsloth/llama-3-8b-Instruct-bnb-4bit"

In [6]:
TASK = "ari"

In [7]:
OUTPUT_DIR = os.path.join(cdcp_dir, "finetuned_models", f"""CDCP_{TASK}_{BASE_MODEL.split("/")[1]}""")

In [8]:
OUTPUT_DIR

'/Utilisateurs/umushtaq/am_work/coling_2025/cdcp/finetuned_models/CDCP_ari_llama-3-8b-Instruct-bnb-4bit'

In [9]:
NB_EPOCHS = 5

## Post-processing

In [10]:
cdcp_dataset = load_dataset("DFKI-SLT/cdcp", trust_remote_code=True)

In [11]:
cdcp_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'propositions', 'relations'],
        num_rows: 580
    })
    test: Dataset({
        features: ['id', 'text', 'propositions', 'relations'],
        num_rows: 150
    })
})

In [50]:
cdcp_dataset['test'][1]

{'id': '00196',
 'text': 'When alleged debtors are served with state court summonses, they are not always comprehensible to laypersons. Any requirements to file papers to avoid default judgment The date of any scheduled hearing and procedures for changing the date Local and online sources of information for pro se defendants, and possibly local non-profit advice organizations. That the debtor may wish to consider bankruptcy if they cannot pay their debts.',
 'propositions': {'start': [0, 109, 167, 238, 353],
  'end': [109, 167, 238, 353, 433],
  'label': [4, 1, 1, 1, 1],
  'url': ['', '', '', '', '']},
 'relations': {'head': [], 'tail': [], 'label': []}}

In [27]:
nr_acs_l = []

for sample in cdcp_dataset['test']:
    nr_acs_l.append(len(sample['propositions']['label']))

In [29]:
len(nr_acs_l)

150

In [ ]:
nr_acs_l

In [113]:
with open(os.path.join(OUTPUT_DIR, f"""CDCP_{TASK}_results_{NB_EPOCHS}.pickle"""), "rb") as fh:
        
        results = pickle.load(fh)

In [114]:
grounds = results["ground_truths"]

In [115]:
grounds = [json.loads(x)["list_argument_relations"] for x in grounds]

In [116]:
preds = results["predictions"]

In [117]:
preds = [x["content"] for x in preds]

In [118]:
preds = [json.loads(x)["list_argument_relations"] for x in preds]

In [119]:
len(grounds), len(preds)

(150, 150)

In [120]:
for i,(x,y) in enumerate(zip(grounds, preds)):
    
    if len(x) != len(y):
            
        print(i)

0
4
5
6
7
9
10
16
17
20
21
25
26
27
28
30
37
38
39
40
41
43
46
47
48
50
51
53
54
55
57
58
60
61
62
64
66
67
71
73
74
75
76
77
78
79
80
81
83
84
85
86
87
88
89
90
92
94
96
99
100
102
103
104
105
106
107
110
114
116
119
121
124
125
130
133
136
137
138
139
141
142
145
146
148
149


In [121]:
def process_grounds(grounds_l, nr_acs_l):

    pairs_l = []
    
    for idx, ac_count in enumerate(nr_acs_l):
        for i in range(ac_count):
            for j in range(ac_count):

                if i != j:
                    pair = [i, j]
                    if pair in grounds_l[idx]:
                        pairs_l.append([i, j, "Rel"])
                    else:
                        pairs_l.append([i, j, "N-Rel"])

    return pairs_l

In [122]:
def process_preds(preds_l, nr_acs_l):

    pairs_l = []
    
    for idx, ac_count in enumerate(nr_acs_l):
        for i in range(ac_count):
            for j in range(ac_count):

                if i != j:
                    pair = [i, j]
                    if pair in preds_l[idx]:
                        pairs_l.append([i, j, "Rel"])
                    else:
                        pairs_l.append([i, j, "N-Rel"])

    return pairs_l

In [123]:
final_grounds = process_grounds(grounds, nr_acs_l)

In [124]:
final_grounds

[[0, 1, 'N-Rel'],
 [0, 2, 'N-Rel'],
 [1, 0, 'N-Rel'],
 [1, 2, 'N-Rel'],
 [2, 0, 'N-Rel'],
 [2, 1, 'Rel'],
 [0, 1, 'N-Rel'],
 [0, 2, 'N-Rel'],
 [0, 3, 'N-Rel'],
 [0, 4, 'N-Rel'],
 [1, 0, 'N-Rel'],
 [1, 2, 'N-Rel'],
 [1, 3, 'N-Rel'],
 [1, 4, 'N-Rel'],
 [2, 0, 'N-Rel'],
 [2, 1, 'N-Rel'],
 [2, 3, 'N-Rel'],
 [2, 4, 'N-Rel'],
 [3, 0, 'N-Rel'],
 [3, 1, 'N-Rel'],
 [3, 2, 'N-Rel'],
 [3, 4, 'N-Rel'],
 [4, 0, 'N-Rel'],
 [4, 1, 'N-Rel'],
 [4, 2, 'N-Rel'],
 [4, 3, 'N-Rel'],
 [0, 1, 'N-Rel'],
 [1, 0, 'N-Rel'],
 [0, 1, 'N-Rel'],
 [1, 0, 'N-Rel'],
 [0, 1, 'N-Rel'],
 [0, 2, 'N-Rel'],
 [0, 3, 'N-Rel'],
 [0, 4, 'N-Rel'],
 [1, 0, 'N-Rel'],
 [1, 2, 'N-Rel'],
 [1, 3, 'N-Rel'],
 [1, 4, 'N-Rel'],
 [2, 0, 'N-Rel'],
 [2, 1, 'N-Rel'],
 [2, 3, 'N-Rel'],
 [2, 4, 'N-Rel'],
 [3, 0, 'N-Rel'],
 [3, 1, 'N-Rel'],
 [3, 2, 'N-Rel'],
 [3, 4, 'N-Rel'],
 [4, 0, 'N-Rel'],
 [4, 1, 'N-Rel'],
 [4, 2, 'Rel'],
 [4, 3, 'Rel'],
 [0, 1, 'N-Rel'],
 [0, 2, 'N-Rel'],
 [0, 3, 'N-Rel'],
 [0, 4, 'N-Rel'],
 [0, 5, 'N-Rel'],
 [1, 0, 'N-Rel']

In [133]:
final_preds = process_preds(preds, nr_acs_l)

In [134]:
final_preds

[[0, 1, 'N-Rel'],
 [0, 2, 'N-Rel'],
 [1, 0, 'N-Rel'],
 [1, 2, 'N-Rel'],
 [2, 0, 'Rel'],
 [2, 1, 'Rel'],
 [0, 1, 'N-Rel'],
 [0, 2, 'N-Rel'],
 [0, 3, 'N-Rel'],
 [0, 4, 'N-Rel'],
 [1, 0, 'N-Rel'],
 [1, 2, 'N-Rel'],
 [1, 3, 'N-Rel'],
 [1, 4, 'N-Rel'],
 [2, 0, 'N-Rel'],
 [2, 1, 'N-Rel'],
 [2, 3, 'N-Rel'],
 [2, 4, 'N-Rel'],
 [3, 0, 'N-Rel'],
 [3, 1, 'N-Rel'],
 [3, 2, 'N-Rel'],
 [3, 4, 'N-Rel'],
 [4, 0, 'N-Rel'],
 [4, 1, 'N-Rel'],
 [4, 2, 'N-Rel'],
 [4, 3, 'N-Rel'],
 [0, 1, 'N-Rel'],
 [1, 0, 'N-Rel'],
 [0, 1, 'N-Rel'],
 [1, 0, 'N-Rel'],
 [0, 1, 'N-Rel'],
 [0, 2, 'Rel'],
 [0, 3, 'Rel'],
 [0, 4, 'N-Rel'],
 [1, 0, 'N-Rel'],
 [1, 2, 'Rel'],
 [1, 3, 'Rel'],
 [1, 4, 'N-Rel'],
 [2, 0, 'N-Rel'],
 [2, 1, 'N-Rel'],
 [2, 3, 'N-Rel'],
 [2, 4, 'N-Rel'],
 [3, 0, 'N-Rel'],
 [3, 1, 'N-Rel'],
 [3, 2, 'N-Rel'],
 [3, 4, 'N-Rel'],
 [4, 0, 'N-Rel'],
 [4, 1, 'N-Rel'],
 [4, 2, 'N-Rel'],
 [4, 3, 'N-Rel'],
 [0, 1, 'N-Rel'],
 [0, 2, 'N-Rel'],
 [0, 3, 'N-Rel'],
 [0, 4, 'N-Rel'],
 [0, 5, 'N-Rel'],
 [1, 0, 'N-Rel'],
 [1,

In [129]:
final_grounds = [x[2] for x in final_grounds]
final_preds = [x[2] for x in final_preds]

In [130]:
len(final_grounds)

10328

In [131]:
len(final_preds)

10328

In [132]:
print(classification_report(final_grounds, final_preds, digits=3))

              precision    recall  f1-score   support

       N-Rel      0.980     0.984     0.982     10004
         Rel      0.431     0.383     0.405       324

    accuracy                          0.965     10328
   macro avg      0.705     0.683     0.694     10328
weighted avg      0.963     0.965     0.964     10328

